In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ZGlobalLib.gpu_utils import use_gpu
use_gpu(1)

2023-12-25 19:40:37.391649: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-25 19:40:37.817478: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 224 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:68:00.0, compute capability: 7.5


PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')

In [3]:
from histolungs import *

# Test

# Evaluation

In [4]:
def train_evaluate(class_type, resolution,
                   public_directory = 'data/public_dataset/',
                   root_directory = "data/dataset_2_final/", 
                   test_directory = "data/validation_final_septiembre/",
                   pretrain_dir = None,
                   exclude_pd = False
                  ):
    
    if resolution == 'public':
        root_directory = public_directory
    
    resname = resolution if resolution is not None else 'all'
    
    image_paths = get_files(root_directory, resolution=resolution, exclude_pd=exclude_pd)
    class_names, class2int, labels = get_classes_labels(root_directory, image_paths, class_type, exclude_pd=exclude_pd)
    
    num_classes = len(class2int)
    
    display_markdown(f'## Evaluating {resname} resolution, {num_classes} classes, exc pd {exclude_pd}', raw=True)
    
    train_generator, val_generator = get_generators(image_paths, labels, num_classes=num_classes, resolution=resolution)
    class_weights = compute_weights(train_generator)

    model = get_model(num_classes, resolution=resolution)
    
    if pretrain_dir is not None:
        model.load_weights(pretrain_dir)
    
    MODEL_NAME = f'NewEf0_{resname}_{num_classes}_classes_excpd{int(exclude_pd)}'
    RUN_NAME = ''
    log_dir = f'logs/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}/{MODEL_NAME}{RUN_NAME}'
    print(log_dir)
    
    history = train_model(model, train_generator, val_generator, num_classes, class_weights, log_dir)
    
    plot_metrics(history, log_dir)
    
    test_gen = get_test_generator(class2int, resolution, log_dir, exclude_pd=exclude_pd, test_directory=test_directory)
    test_model(model, test_gen, log_dir, class_names)

In [ ]:
for resolution, classes, poco_dif in [('20x', ['macro','micro'], [True, False]), 
                                      ('40x', ['macro','micro'], [True, False]), 
                                      (None, ['macro','micro'], [True, False]), 
                                      ('public', ['macro'], [False])]:
    for class_type in classes:
        for exclude_pd in poco_dif:
            # for pretrain_dir in pretrains:
            train_evaluate(class_type=class_type, resolution=resolution, exclude_pd=exclude_pd)

## Evaluating 20x resolution, 3 classes, exc pd True

  0%|          | 0/34 [00:00<?, ?it/s]

logs/20231219-075847/NewEf0_20x_3_classes_excpd1


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

2023-12-19 07:58:50.910008: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
